<a href="https://colab.research.google.com/github/Minakshi85/Spam-Filter-for-Quora-Questions/blob/master/P3_Spam_filter_for_Quora_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
#from keras.layers.recurrent import LSTM,GRU
#from keras.layers.core import Dense, Activation, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score
#from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [2]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv?dl=0

--2024-03-07 02:59:06--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv [following]
--2024-03-07 02:59:06--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucebb10a6be9c8fb2ef95e35753c.dl.dropboxusercontent.com/cd/0/inline/COnnmy8MSAuLRA20wMd5PpgGtLV7U-r9OR0qLKYqXYN_f_nl5Be9b_0NQyCVFpzQHex3ZEuIqi6hIE1czrWhgVDOA4MErLzPWWG9CQa6BpHD7p_RDNiHSqzE82-if2qCNCg/file# [following]
--2024-03-07 02:59:07--  https://ucebb10a6be9c8fb2ef95e35753c.dl.dropboxusercontent.com/cd/0/inline/COnnmy8MSAuLRA20wMd5PpgGtLV7U-r9OR0qLKYqXYN_f_

In [3]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2024-03-07 02:59:23--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2024-03-07 02:59:23--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2024-03-07 02:59:23--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]


In [4]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [3]:
embedding_index ={}

f = open('/content/glove.42B.300d.txt', encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype ='float32')
  embedding_index[word] = coefs
f.close()

In [4]:
question = pd.read_csv(r'/content/train.csv?dl=0')

In [5]:
print(question.shape)
question.head()

(1306122, 3)


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [6]:
X = question['question_text']
Y = question['target']
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1044897,), (1044897,), (261225,), (261225,))

In [7]:
y_test.value_counts()

0    244957
1     16268
Name: target, dtype: int64

In [8]:
#!pip install imbalanced-learn
import imblearn
from sklearn.utils.class_weight import compute_class_weight

In [9]:

class_weight = compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train)
#class_weight = imblearn.combine.compute_class_weight('balanced', np.unique(y_train), y_train)

#print(class_weight)
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: 0.5329176675796012, 1: 8.094705772985034}

In [10]:
x_train

143878                   What would an INTP parent act like?
1069070    Why are visions the same while their names are...
301067     What is the national committee on sedimentatio...
479511     How do I measure the similarity between two fM...
564268             Did you ever rebel against your parent's?
                                 ...                        
502265     What are the different types of risk that cert...
27783      How did the German economic miracle come about...
4207       Will the negative consequences of modernizatio...
699635     Do you think that the most important duty of w...
891487     What is your favorite game that you played tha...
Name: question_text, Length: 1044897, dtype: object

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [13]:
print(np.percentile(sent_lens,99))
np.percentile(sent_lens,95)

45.0


31.0

In [14]:
max_len=45

tk=Tokenizer(char_level=False,split=' ')
tk.fit_on_texts(x_train)
print(x_train.shape)

(1044897,)


In [15]:

x_train.tail(10)

243482     What reasons are there to think Carter was a b...
304328     What is the politically correct spelling: Ass-...
606648     What are the responsibilities of the National ...
1193004            Do you know anyone that hates technology?
140065     How do I do Dhyana according to Acharya Rajneesh?
502265     What are the different types of risk that cert...
27783      How did the German economic miracle come about...
4207       Will the negative consequences of modernizatio...
699635     Do you think that the most important duty of w...
891487     What is your favorite game that you played tha...
Name: question_text, dtype: object

In [16]:
seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)
print(len(seq_test), len(seq_train))
seq_train[0]

261225 1044897


[2, 35, 29, 6996, 1628, 612, 42]

In [17]:
vocab_size=len(tk.word_index)
vocab_size

196159

In [18]:
seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 45), (261225, 45))

In [19]:
# creating our own embedding matrix to bring down the size to 300
# we'll use 300 D vector representation of the words from pretrained embedding index
# that we downloaded

embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embedding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

# if there are specific words which are not present in pretrained embedding
# their weights will remain 0. if there are too many such words
# then you should probably not use pretrained embeddings


In [20]:
# Now while creating the model for text data , we'll keep embedding layer weights
# non-trainable as we are supplying those values from our end

inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
#lstm_layer1 = LSTM(64)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
#lstm_layer2 = Dense(32,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
#lstm_layer3 = Dense(16,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
#dense1 = Dense(1,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [21]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 45)]              0         
                                                                 
 embedding (Embedding)       (None, 45, 300)           58848000  
                                                                 
 lstm (LSTM)                 (None, 512)               1665024   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896 

In [22]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,
                             save_best_only=True)

In [23]:
earlystop = EarlyStopping(monitor='val_loss', patience=1,
                          verbose=1)

In [24]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [25]:
model.fit(seq_train_matrix, y_train, batch_size=1000, epochs=1, validation_data=(seq_test_matrix, y_test),class_weight={0: 0.5328573351855138, 1: 8.108651115146436},callbacks = [earlystop, checkpoint])

1045/1045 [==============================] - ETA: 0s - loss: 0.2964 - accuracy: 0.8746 
Epoch 1: val_loss improved from inf to 0.25648, saving model to /content/weights-01-0.2565.h5
1045/1045 [==============================] - 12940s 12s/step - loss: 0.2964 - accuracy: 0.8746 - val_loss: 0.2565 - val_accuracy: 0.9021


In [26]:
pred = model.predict([seq_test_matrix], batch_size=1000, verbose=1)

262/262 [==============================] - 1078s 4s/step


In [27]:
pred.shape

(261225, 1)

In [ ]:
#model.fit(seq_train_matrix,y_train,
             # epochs=1,
              #class_weight={0: 0.5328573351855138, 1: 8.108651115146436},
              #batch_size=32,
              #validation_data=(seq_test_matrix,y_test),
              #callbacks = [earlystop, checkpoint])

In [28]:
best_model = load_model('/content/weights-01-0.2565.h5')
#best_model = load_model('/content/model_weights.h5')

In [29]:
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 45)]              0         
                                                                 
 embedding (Embedding)       (None, 45, 300)           58848000  
                                                                 
 lstm (LSTM)                 (None, 512)               1665024   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896 

In [30]:
p = model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

8164/8164 [==============================] - 1623s 199ms/step


0.9573496153792217

In [31]:
p

array([[0.240111  ],
       [0.00704504],
       [0.07660869],
       ...,
       [0.54234844],
       [0.04158759],
       [0.03751912]], dtype=float32)

In [ ]:
# Saving the model
#from tensorflow.keras import models
#from tensorflow.keras.models import model_from_json
#model_json = model.to_json()
#with open("model.json", "w") as json_file:
  #json_file.write(model_json)
#model.save_weights('model_weights.h5',overwrite = True)

In [ ]:
#from tensorflow.keras.models import model_from_json

#json_file = open('model.json', 'r')

#loaded_model_json = json_file.read()
#json_file.close()

#loaded_model = model_from_json(loaded_model_json)

#loaded_model.summary()

#loaded_model.load_weights('model_weights.h5')
